# Análisis de Encuesta Nacional de Calidad de Vida - ECV

## Integrantes - Grupo 15

- Sara Sofia Cárdenas Rodríguez - 202214907
- Daniel Felipe Diaz Moreno - 202210773
- Juan Sebastián Urrea López - 201914710


## Los datos

Se trabaja con un conjunto de datos para cada uno de los 4 años requeridos.


## 1. Carga y Manipulación básica de los Datos

In [1]:
# Importaciones de bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import sys
import re
import string
import unicodedata

# Importaciones de joblib
from joblib import dump, load

# Importaciones de sklearn
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn import tree
from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
salud2017 = pd.read_csv("Salud2017.csv", delimiter=";")
salud2018 = pd.read_csv("Salud2018.csv", delimiter=";")
salud2021 = pd.read_csv("Salud2021.csv", delimiter=",")
salud2022 = pd.read_csv("Salud2022.csv", delimiter=";")

print(f"Las dimensiones de salud2017 son: {salud2017.shape}")
print(f"Las dimensiones de salud2018 son: {salud2018.shape}")
print(f"Las dimensiones de salud2021 son: {salud2021.shape}")
print(f"Las dimensiones de salud2022 son: {salud2022.shape}")

Las dimensiones de salud2017 son: (26500, 103)
Las dimensiones de salud2018 son: (283012, 98)
Las dimensiones de salud2021 son: (257589, 145)
Las dimensiones de salud2022 son: (251925, 151)


Las variables candidatas descritas en el informe son las siguientes:

- ¿Está afiliado(a) (cotizante o beneficiario(a)) a alguna entidad de seguridad social en salud? (Entidad Promotora de Salud - EPS o Entidad Promotora de Salud Subsidiada -EPS-S ) (P6090) 

- ¿Por qué razón principal no está afiliado(a) a una entidad de seguridad social en salud? (Entidad Promotora de Salud - EPS o Entidad Promotora de Salud Subsidiada - EPS-S) (P768) 

- ¿A cuál de los siguientes regímenes de seguridad social en salud está afiliado: (P6100) 

- En general, considera que la calidad del servicio de la entidad de seguridad social en salud a la que ______ está afiliado(a) es: (P6181) 

- ¿Está embarazada actualmente o ha tenido hijos? (P5672) 

- Si está embarazada actualmente, ¿asiste a control prenatal? (P5694) 

- ¿Durante este embarazo le han formulado suplementos vitamínicos (Sulfato ferroso, ácido fólico, calcio, etc.)? (P5695) 

Se procederá a evidenciar si estos atributos se encuentran en los cuatro archivos de datos que se tienen disponibles.

In [3]:
variablesABuscar = set(["P6090", "P768", "P6100", "P6181", "P5672", "P5694", "P5695"])

columnasSalud2017 = [col for col in salud2017.columns if col in variablesABuscar]
columnasSalud2018 = [col for col in salud2018.columns if col in variablesABuscar]
columnasSalud2021 = [col for col in salud2021.columns if col in variablesABuscar]
columnasSalud2022 = [col for col in salud2022.columns if col in variablesABuscar]

columnasSalud2017.sort()
columnasSalud2018.sort()
columnasSalud2021.sort()
columnasSalud2022.sort()

print(f"Las columnas de salud2017 son: {columnasSalud2017}")
print(f"Las columnas de salud2018 son: {columnasSalud2018}")
print(f"Las columnas de salud2021 son: {columnasSalud2021}")
print(f"Las columnas de salud2022 son: {columnasSalud2022}")

columnasEnComun = set(columnasSalud2017).intersection(set(columnasSalud2018)).intersection(set(columnasSalud2021)).intersection(set(columnasSalud2022))

print(f"Las columnas en común son: {columnasEnComun}")
print(f"Las columnas que no estan en todos los datasets son: {variablesABuscar - columnasEnComun}")

Las columnas de salud2017 son: ['P5672', 'P5694', 'P5695', 'P6090', 'P6100', 'P6181', 'P768']
Las columnas de salud2018 son: ['P5672', 'P5694', 'P5695', 'P6090', 'P6100', 'P6181', 'P768']
Las columnas de salud2021 son: ['P5694', 'P6090', 'P6100', 'P6181', 'P768']
Las columnas de salud2022 son: ['P5694', 'P6090', 'P6100', 'P6181', 'P768']
Las columnas en común son: {'P768', 'P6090', 'P6181', 'P5694', 'P6100'}
Las columnas que no estan en todos los datasets son: {'P5695', 'P5672'}


Con base en esto, se puede observar que no se encentran los siguientes atributos.

- ¿Durante este embarazo le han formulado suplementos vitamínicos (Sulfato ferroso, ácido fólico, calcio, etc.)? (P5695)
- ¿Está embarazada actualmente o ha tenido hijos? (P5672)

Al consultar en los dos archivos si existia alguna manera de reemplazar estos datos, no se encontró manera para P5695. Sin embargo, si fue posible encontrar el atributo 
¿ha estado embarazada alguna vez en su vida? (P3335), la cual funge el mismo rol que P5672. Por lo tanto, se procederá a reemplazar P5672 por P3335 para estos dos archivos y se decidira no tener en cuenta P5695 en el analisis de datos

In [4]:
columnasSalud2017.remove("P5695")
columnasSalud2018.remove("P5695")
columnasSalud2021.append("P3335")
columnasSalud2022.append("P3335")

Asimismo, para facilitiar su análisis, se procederá a cambiar los identificadores de las columnas por nombres más descriptivos.

In [5]:
idANombre = {
    'P5694': '¿Asiste a control prenatal?',
    'P6090': '¿Está afiliado?',
    'P6100': '¿A qué régimen está afiliado?',
    'P6181': '¿Cómo es la calidad del servicio?',
    'P768': '¿Por qué no está afiliado?',
    'P5672': '¿Está embarazada?',
    'P3335': '¿Está embarazada?',
}
columnasSalud2017 = [idANombre[col] for col in columnasSalud2017]
columnasSalud2018 = [idANombre[col] for col in columnasSalud2018]
columnasSalud2021 = [idANombre[col] for col in columnasSalud2021]
columnasSalud2022 = [idANombre[col] for col in columnasSalud2022]


salud2017.rename(columns=idANombre, inplace=True)
salud2018.rename(columns=idANombre, inplace=True)
salud2021.rename(columns=idANombre, inplace=True)
salud2022.rename(columns=idANombre, inplace=True)



Con esto, a continuación se presenta una muestra para cada uno de los archivos de datos disponibles.

In [6]:
salud2017[columnasSalud2017].sample(10)

,¿Está embarazada?,¿Asiste a control prenatal?,¿Está afiliado?,¿A qué régimen está afiliado?,¿Cómo es la calidad del servicio?,¿Por qué no está afiliado?
4073,,,1,1,2,
23387,,,1,1,2,
14682,,,1,1,2,
20413,,,1,1,1,
23657,,,1,1,2,
23667,,,1,1,2,
4683,2,,1,3,2,
15927,,,1,3,2,
14178,,,1,1,1,
18694,,,1,3,3,


In [7]:
salud2018[columnasSalud2018].sample(10)

,¿Está embarazada?,¿Asiste a control prenatal?,¿Está afiliado?,¿A qué régimen está afiliado?,¿Cómo es la calidad del servicio?,¿Por qué no está afiliado?
272487,,,1,1,2,
69383,,,1,3,3,
55874,,,1,1,2,
129890,2,,1,3,3,
193922,,,1,3,2,
37558,,,1,3,2,
212079,,,1,1,2,
17080,,,1,3,2,
194708,,,2,,,5
26127,2,,1,3,2,


In [8]:
salud2021[columnasSalud2021].sample(10)

,¿Asiste a control prenatal?,¿Está afiliado?,¿A qué régimen está afiliado?,¿Cómo es la calidad del servicio?,¿Por qué no está afiliado?,¿Está embarazada?
113512,NaN,1,1.0,3.0,NaN,2.0
104829,NaN,1,3.0,3.0,NaN,2.0
123346,NaN,1,1.0,2.0,NaN,NaN
197379,NaN,1,3.0,1.0,NaN,NaN
235048,NaN,1,1.0,2.0,NaN,NaN
82591,NaN,1,1.0,2.0,NaN,2.0
128209,NaN,1,1.0,2.0,NaN,2.0
128992,NaN,1,1.0,2.0,NaN,1.0
217200,NaN,1,1.0,1.0,NaN,NaN
115537,NaN,1,3.0,1.0,NaN,NaN


In [9]:
salud2022[columnasSalud2022].sample(10)

,¿Asiste a control prenatal?,¿Está afiliado?,¿A qué régimen está afiliado?,¿Cómo es la calidad del servicio?,¿Por qué no está afiliado?,¿Está embarazada?
206803,NaN,1,3.0,2.0,NaN,NaN
24936,NaN,1,1.0,2.0,NaN,NaN
199267,NaN,1,3.0,9.0,NaN,NaN
10321,NaN,1,3.0,2.0,NaN,NaN
16231,NaN,1,3.0,2.0,NaN,1.0
28797,NaN,1,3.0,2.0,NaN,1.0
122723,NaN,1,3.0,2.0,NaN,NaN
68751,NaN,1,3.0,2.0,NaN,1.0
110516,NaN,1,1.0,3.0,NaN,2.0
105712,NaN,1,3.0,2.0,NaN,2.0


Observamos que muchos datos son nulos, esto se debe a que varias preguntas son mutuamente excluyentes, y otras son dependientes de otras

Finalmente, se muestra el tipo de dato para cada atributo por cada archivo de datos.

In [10]:
salud2017[columnasSalud2017].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26500 entries, 0 to 26499
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   ¿Está embarazada?                  26500 non-null  object
 1   ¿Asiste a control prenatal?        26500 non-null  object
 2   ¿Está afiliado?                    26500 non-null  int64 
 3   ¿A qué régimen está afiliado?      26500 non-null  object
 4   ¿Cómo es la calidad del servicio?  26500 non-null  object
 5   ¿Por qué no está afiliado?         26500 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


In [11]:
salud2018[columnasSalud2018].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283012 entries, 0 to 283011
Data columns (total 6 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   ¿Está embarazada?                  283012 non-null  object
 1   ¿Asiste a control prenatal?        283012 non-null  object
 2   ¿Está afiliado?                    283012 non-null  int64 
 3   ¿A qué régimen está afiliado?      283012 non-null  object
 4   ¿Cómo es la calidad del servicio?  283012 non-null  object
 5   ¿Por qué no está afiliado?         283012 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.0+ MB


In [12]:
salud2021[columnasSalud2021].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257589 entries, 0 to 257588
Data columns (total 6 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ¿Asiste a control prenatal?        1439 non-null    float64
 1   ¿Está afiliado?                    257589 non-null  int64  
 2   ¿A qué régimen está afiliado?      240869 non-null  float64
 3   ¿Cómo es la calidad del servicio?  240152 non-null  float64
 4   ¿Por qué no está afiliado?         15579 non-null   float64
 5   ¿Está embarazada?                  113214 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 11.8 MB


In [13]:
salud2022[columnasSalud2022].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251925 entries, 0 to 251924
Data columns (total 6 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ¿Asiste a control prenatal?        1302 non-null    float64
 1   ¿Está afiliado?                    251925 non-null  int64  
 2   ¿A qué régimen está afiliado?      239356 non-null  float64
 3   ¿Cómo es la calidad del servicio?  238802 non-null  float64
 4   ¿Por qué no está afiliado?         11496 non-null   float64
 5   ¿Está embarazada?                  111290 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 11.5 MB


Vemos que hay problemas de consistencia en los tipos de datos, pues los atributos enteros de 2021 y 2022 aparecen como textos en 2017 y 2018. Al usar la siguiente linea de codigo, se ve que los valores nulos se toman como " ", por tal razon se procedera a reemplazar estos valores por valores nulos

In [14]:
salud2017["¿Asiste a control prenatal?"].unique()

array([' ', '3', '2', '1'], dtype=object)

In [15]:
for col in columnasSalud2017:
    salud2017[col] = pd.to_numeric(salud2017[col], errors="coerce")

for col in columnasSalud2018:
    salud2018[col] = pd.to_numeric(salud2018[col], errors="coerce")

In [16]:
salud2017[columnasSalud2017].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26500 entries, 0 to 26499
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ¿Está embarazada?                  8353 non-null   float64
 1   ¿Asiste a control prenatal?        4307 non-null   float64
 2   ¿Está afiliado?                    26500 non-null  int64  
 3   ¿A qué régimen está afiliado?      25243 non-null  float64
 4   ¿Cómo es la calidad del servicio?  25187 non-null  float64
 5   ¿Por qué no está afiliado?         1214 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 1.2 MB


In [17]:
salud2018[columnasSalud2018].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283012 entries, 0 to 283011
Data columns (total 6 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ¿Está embarazada?                  85837 non-null   float64
 1   ¿Asiste a control prenatal?        48340 non-null   float64
 2   ¿Está afiliado?                    283012 non-null  int64  
 3   ¿A qué régimen está afiliado?      264794 non-null  float64
 4   ¿Cómo es la calidad del servicio?  264198 non-null  float64
 5   ¿Por qué no está afiliado?         17166 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 13.0 MB


In [18]:
salud2017['año'] = 2017
salud2018['año'] = 2018
salud2021['año'] = 2021
salud2022['año'] = 2022

salud_df = pd.concat([salud2017[columnasSalud2017 + ["año"]], salud2018[columnasSalud2018+ ["año"]], salud2021[columnasSalud2021+ ["año"]], salud2022[columnasSalud2022+ ["año"]]], axis=0)
salud_df.reset_index(drop=True, inplace=True)
salud_df.sample(10)

,¿Está embarazada?,¿Asiste a control prenatal?,¿Está afiliado?,¿A qué régimen está afiliado?,¿Cómo es la calidad del servicio?,¿Por qué no está afiliado?,año
783415,1.0,NaN,1,3.0,2.0,NaN,2022
59107,NaN,NaN,1,1.0,3.0,NaN,2018
734911,NaN,NaN,2,NaN,NaN,7.0,2022
623967,1.0,NaN,1,3.0,2.0,NaN,2022
479129,2.0,NaN,1,3.0,2.0,NaN,2021
617014,1.0,NaN,2,NaN,NaN,6.0,2022
319580,NaN,NaN,1,1.0,2.0,NaN,2021
679560,NaN,NaN,1,3.0,2.0,NaN,2022
318967,1.0,NaN,1,3.0,2.0,NaN,2021
767688,2.0,NaN,1,3.0,2.0,NaN,2022


## 2. Entendimiento de los datos

## 2.1. Análisis de calidad de datos





## 2.1.1. Análisis de completitud
A continuación, se presentan los porcentajes de completitud e incompletitud para cada uno de los atributos.

In [19]:
porcentajeIncompletitudAtributo =  (100 * salud_df.isnull().sum() / salud_df.shape[0]).sort_values(ascending=False)
pd.DataFrame({
    "Atributo": porcentajeIncompletitudAtributo.index,
    "Completitud (%)": 100 - porcentajeIncompletitudAtributo.values,
    "Incompletitud (%)": porcentajeIncompletitudAtributo.values
})

,Atributo,Completitud (%),Incompletitud (%)
0,¿Por qué no está afiliado?,5.549885,94.450115
1,¿Asiste a control prenatal?,6.762667,93.237333
2,¿Está embarazada?,38.911341,61.088659
3,¿Cómo es la calidad del servicio?,93.811308,6.188692
4,¿A qué régimen está afiliado?,94.046099,5.953901
5,¿Está afiliado?,100.000000,0.000000
6,año,100.000000,0.000000


Si bien se ve una alta incompletitud, los atributos más "incompletos" son aquellos que de alguna u otra manera estan limitados por otros atributos, lo que es la causa de su incompletitud.

## 2.1.2. Análisis de unicidad
A continuación, se buscará determinar cuántas filas duplicadas hay en el dataset

In [20]:
registrosDuplicados = salud_df.duplicated()
totalDuplicados = registrosDuplicados.sum()
totalOpiniones = salud_df.shape[0]
porcentajeDuplicados = 100.0 * totalDuplicados / totalOpiniones
print(f"Se cuenta con {totalDuplicados} registros duplicados ({porcentajeDuplicados:.2f}%)")



Se cuenta con 818581 registros duplicados (99.95%)


Los datos presentan un alto porcentaje de duplicados, lo cual se debe a que se cuenta con un subconjunto muy limitado de los atributos totales, teniendo cada uno de estos valores una lista de posibles valores corta y que estan muy relacionadas entre si. Por estas razones, no se deberia realizar preparacion de datos con respecto a problemas de unicidad.

## 2.1.3. Análisis de validez
De acuerdo al diccionario de datos del año 2022, se deberian tener los siguientes posibles valores por cada atributo:

** ¿Está embarazada? **
1 Si
2 No

** ¿Asiste a control prenatal? **
1 Sí
2 No
3 No está embarazada

** ¿Está afiliado? **
1 Si
2 No
9 No sabe, no informa


** ¿A qué regimen está afiliado? **
1 Contributivo (eps)
2 Especial (fuerzas armadas, ecopetrol, universidades públicas, magisterio)
3 Subsidiado (eps-s)
9 No sabe, no informa

** ¿Cómo es la calidad del servicio? **
1 Muy buena
2 Buena
3 Mala
4 Muy mala
9 No sabe

** ¿Por qué no está afiliado? **
1 Por falta de dinero
2 Muchos trámites
3 No le interesa o descuido
4 No sabe que debe afiliarse
5 No está vinculado/a laboralmente a una empresa o entidad (Usted o la persona de la que es beneficiario/a)
6 Está en trámite de afiliación
7 Problemas con el Sisben (no lo/a han visitado, afiliado/a en otro municipio, lo/a desvincularon, le asignaron puntaje alto)
8 Falta de documentación
9 Otra razón










In [21]:
for col in salud_df:
    print(col, ":", sorted(salud_df[col].unique()))



¿Está embarazada? : [np.float64(nan), np.float64(1.0), np.float64(2.0)]
¿Asiste a control prenatal? : [np.float64(nan), np.float64(1.0), np.float64(2.0), np.float64(3.0)]
¿Está afiliado? : [np.int64(1), np.int64(2), np.int64(9)]
¿A qué régimen está afiliado? : [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(nan), np.float64(9.0)]
¿Cómo es la calidad del servicio? : [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(9.0), np.float64(nan)]
¿Por qué no está afiliado? : [np.float64(nan), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0)]
año : [np.int64(2017), np.int64(2018), np.int64(2021), np.int64(2022)]


De acuerdo con los resultados anteriores, no se debería realizar preparación de datos con problemas de validez, pues todos los valores de los atributos son aceptables.

## 2.1.4. Análisis de consistencia

El único problema de consistencia se mencionó en la sección 1, donde en una pareja de conjuntos de datos se tenia el atributo "¿ha estado embarazada alguna vez en su vida? (P3335)" mientras que otra pareja tenia el atributo 
"¿Está embarazada actualmente o ha tenido hijos? (P5672)" que en escencia hacian referencia a lo mismo. No se identificaron mas tareas a realizar por parte de problemas de consistencias.

# 3. Resumen de los cambios realizados